In [5]:
import featuretools as ft

In [6]:
 data = ft.demo.load_mock_customer()

In [7]:
customers_df = data["customers"]

In [8]:
customers_df

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [9]:
sessions_df = data["sessions"]

In [10]:
sessions_df

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30
5,6,1,tablet,2014-01-01 01:23:25
6,7,3,tablet,2014-01-01 01:39:40
7,8,4,tablet,2014-01-01 01:55:55
8,9,1,desktop,2014-01-01 02:15:25
9,10,2,tablet,2014-01-01 02:31:40


In [11]:
transactions_df = data["transactions"]

In [12]:
transactions_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount
264,40,20,2014-01-01 04:46:00,5,53.22
19,370,2,2014-01-01 00:20:35,1,106.99
314,186,23,2014-01-01 05:40:10,5,128.26
290,380,21,2014-01-01 05:14:10,5,57.09
379,261,28,2014-01-01 06:50:35,1,133.71


First, we specify a dictionary with all the entities in our dataset.

In [14]:
entities = {
    "customers:" (customers_df, "customer_id"),
    "sessions:" (sessions_df, "sessions_id", "sessions_start"),
    "transactions:" (transactions_df, "transactions_id", "transactions_time")
}

TypeError: 'str' object is not callable

In [15]:
 entities = {
   ...:    "customers" : (customers_df, "customer_id"),
   ...:    "sessions" : (sessions_df, "session_id", "session_start"),
   ...:    "transactions" : (transactions_df, "transaction_id", "transaction_time")
   ...: }
   ...: 

Second, we specify how the entities are related. When two entities have a one-to-many relationship, we call the “one” enitity, the “parent entity”. A relationship between a parent and child is defined like this:

In [16]:
(parent_entity, parent_variable, child_entity, child_variable)

NameError: name 'parent_entity' is not defined

In this dataset we have two relationships

In [17]:
 relationships = [("sessions", "session_id", "transactions", "session_id"),
   ....:                  ("customers", "customer_id", "sessions", "customer_id")]

# Run Deep Feature Synthesis
A minimal input to DFS is a set of entities, a list of relationships, and the “target_entity” to calculate features for. The ouput of DFS is a feature matrix and the corresponding list of feature defintions.

Let’s first create a feature matrix for each customer in the data

In [18]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
   ....:                                                  relationships=relationships,
   ....:                                                  target_entity="customers")

In [19]:
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start))
customer_id,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,...,4,1,1,1,1,4,1,2014,1,2
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,...,4,1,1,1,1,3,1,2014,1,2
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,...,4,1,1,1,1,1,1,2014,1,2
4,60091,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,...,5,1,1,1,1,1,1,2014,1,2
5,60091,6,3,mobile,6349.66,44.095630,149.02,-0.025941,7.55,80.375443,...,5,1,1,1,1,3,1,2014,1,2


We now have dozens of new features to describe a customer’s behavior.

# Change target entity
One of the reasons DFS is so powerful is that it can create a feature matrix for any entity in our data. For example, if we wanted to build features for sessions.

In [20]:
feature_matrix_sessions, features_defs = ft.dfs(entities=entities,
   ....:                                                 relationships=relationships,
   ....:                                                 target_entity="sessions")

In [22]:
feature_matrix_sessions.head()

,customer_id,device,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),...,customers.NUM_UNIQUE(transactions.product_id),customers.MODE(transactions.product_id),customers.DAY(join_date),customers.DAY(date_of_birth),customers.YEAR(join_date),customers.YEAR(date_of_birth),customers.MONTH(join_date),customers.MONTH(date_of_birth),customers.WEEKDAY(join_date),customers.WEEKDAY(date_of_birth)
session_id,,,,,,,,,,,,,,,,,,,,,
1,2,desktop,1229.01,41.600976,141.66,0.295458,20.91,76.813125,16,5,...,5,4,15,18,2012,1986,4,8,6,0
2,5,mobile,746.96,45.893591,135.25,-0.160550,9.32,74.696000,10,5,...,5,5,17,28,2010,1984,7,7,5,5
3,4,mobile,1329.00,46.240016,147.73,-0.324012,8.70,88.600000,15,5,...,5,2,8,15,2011,2006,4,8,4,1
4,1,mobile,1613.93,40.187205,129.00,0.234349,6.29,64.557200,25,5,...,5,4,17,18,2011,1994,4,7,6,0
5,4,mobile,777.02,48.918663,139.20,0.336381,7.43,70.638182,11,5,...,5,2,8,15,2011,2006,4,8,4,1
